Equipo:
Abigail Velasco García A01638095
Natalia Velasco García A01638095

In [5]:
#Abigail Velasco García A01638095
#Natalia Velasco García A01638047

# Model design
import agentpy as ap
import time

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import IPython

In [6]:
moveC = 0
start_time = 0
end_time = 0

class TileModel(ap.Model):

    def setup(self):
        # Create agents (dTiles)
        n_dTiles = int(self.p['dTiles density'] * (self.p.size['X']) * (self.p.size['Y']))
        dTiles = self.dTiles_agents = ap.AgentList(self, n_dTiles)

        # Create grid (room)
        self.room = ap.Grid(self, (self.p['size']['X'], self.p['size']['Y']), track_empty=True)
        self.room.add_agents(dTiles, random=True, empty=True)

        # Initiate a dynamic variable for tiles 
        # Condition 0: Dirty, 1: Clean

        self.dTiles_agents.condition = 0
        

        # Start cleaning from [1, 1]
        # Create agents (robots)

        robots = self.robot_agents = ap.AgentList(self, self.p['nRobots'])
        robotsPosition = [(1,1)] * self.p['nRobots']
        
        self.robot_agents.condition = 2

        self.room.add_agents(robots, robotsPosition, empty=True)
    
    global start_time
    start_time = time.time()
    def step(self):
        
        global moveC

        # Select dirty tiles
        dTiles = self.dTiles_agents.select(self.dTiles_agents.condition == 0)
        
        # Clean floor

        for robot in self.robot_agents:
            neighborList = self.room.neighbors(robot).to_list()
            neighbor = self.random.choice(neighborList)
            if((time.time() - start_time >=  self.p['time'])):
                self.stop()
            if neighbor.condition == 0:
                neighbor.condition = 1
            elif neighbor.condition != 2:
                newPosRobot = self.room.positions[neighbor]
                self.room.move_to(robot, newPosRobot)
                moveC += 1
                
        if len(dTiles) == 0:
            self.stop()
    
    def end(self):
        

        global moveC
        global end_time

        # Document a measure at the end of the simulation
        
        end_time = time.time()
        self.report('Simulation running time',
                     end_time - start_time )
        
        dirty_tiles = len(self.dTiles_agents.select(self.dTiles_agents.condition == 0))
        total_tiles = self.p.size['X'] * self.p.size['Y']
        clean_tiles = total_tiles - dirty_tiles
        self.report('Clean tiles %',
            clean_tiles * 100 / total_tiles)
        
        self.report('Robot steps #', moveC)

In [7]:
# Define parameters
parameters = {
    'dTiles density': 0.8, 
    'size': { 'X': 50 , 'Y': 50},
    'nRobots': 30,
    'time': 10
}

In [8]:
model = TileModel(parameters)
results = model.run()
results.reporters

Completed: 8031 steps
Run time: 0:00:09.938879
Simulation finished


,seed,Simulation running time,Clean tiles %,Robot steps #
0,78298319827498184106233213679549908025,10.000837,99.88,230373
